<a href="https://colab.research.google.com/github/Hidenori24/Signate_colab/blob/main/SMBC_GreenChallenge_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## importとシードなどの設定

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
# 日本語フォントを簡単に使う
!pip -q install japanize-matplotlib
import japanize_matplotlib

# シードの設定
np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 84.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Driveへの接続、データの取得

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/ML/Signate_1634/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### train/test/sample_submission の取り込み

In [ ]:
train_df  = pd.read_csv(data_path + 'train.csv', parse_dates=['time'], index_col='time')
df = train_df
test_df   = pd.read_csv(data_path + 'test.csv', parse_dates=['time'], index_col='time')
sample_submission_df = pd.read_csv(data_path + 'sample_submit.csv', header=None)  # header 無し

### 概要確認

In [ ]:
# 概要を確認
print("\n--- 学習データ (train_df) ---")
display(train_df.head())
print(f"shape: {train_df.shape}")

print("\n--- テストデータ (test_df) ---")
display(test_df.head())
print(f"shape: {test_df.shape}")

print("\n--- サンプル提出 (sample_submission_df) ---")
display(sample_submission_df.head())
print(f"shape: {sample_submission_df.shape}")


--- 学習データ (train_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,price_actual,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,64.02,spain_total
2015-01-02 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,58.46,spain_total
2015-01-02 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.70,spain_total
2015-01-02 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.91,spain_total
2015-01-02 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,53.07,spain_total


shape: (26280, 92)

--- テストデータ (test_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_wind_deg,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+01:00,279.0,0.0,3927.0,895.0,189.0,230.0,1069.0,1893.0,7104.0,53.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 01:00:00+01:00,282.0,0.0,3948.0,878.0,177.0,1269.0,1058.0,1024.0,7101.0,52.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 02:00:00+01:00,283.0,0.0,3791.0,890.0,175.0,2197.0,1052.0,888.0,7100.0,52.0,...,0,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 03:00:00+01:00,280.0,0.0,3671.0,881.0,175.0,2965.0,1032.0,645.0,7101.0,53.0,...,40,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 04:00:00+01:00,286.0,0.0,3460.0,861.0,173.0,2705.0,1001.0,661.0,7101.0,53.0,...,30,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total


shape: (8760, 91)

--- サンプル提出 (sample_submission_df) ---


,0,1
0,2018-01-01 00:00:00+01:00,0
1,2018-01-01 01:00:00+01:00,0
2,2018-01-01 02:00:00+01:00,0
3,2018-01-01 03:00:00+01:00,0
4,2018-01-01 04:00:00+01:00,0


shape: (8760, 2)


## データの前処理

In [ ]:
# 基本情報を確認
print(f"データのサイズ: {df.shape}")
print(f"期間: {df.index.min()} から {df.index.max()}")
print(f"⽋損値の合計: {df.isna().sum().sum()}")
# ⽋損値を処理
# 時系列データなので、前⽅補間が適切
#df = df.fillna(method='ffill')
df = df.ffill()
# まだ⽋損がある場合は後⽅補間
#df = df.fillna(method='bfill')
df = df.bfill()
# ⽬的変数の確認
print("\n⽬的変数 'price_actual' の統計:")
print(df['price_actual'].describe())
# 価格の時系列プロット
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['price_actual'])
plt.title('電⼒価格の推移')
plt.ylabel('価格')
plt.grid(True)
plt.tight_layout()
plt.savefig('price_time_series.png')
plt.close()
# 説明変数を準備
# item_IDはスペインの合計データのみ使⽤
df = df[df['item_ID'] == 'spain_total']
# 不要な列を削除
X = df.drop(['price_actual', 'item_ID'], axis=1)
# ⽬的変数
y = df['price_actual']

データのサイズ: (26280, 92)
期間: 2015-01-02 00:00:00+01:00 から 2017-12-31 23:00:00+01:00
⽋損値の合計: 321

⽬的変数 'price_actual' の統計:
count    26280.000000
mean        56.028338
std         14.340356
min          9.330000
25%         47.617500
50%         55.930000
75%         65.192500
max        116.800000
Name: price_actual, dtype: float64


## 特徴量選択と次元削減

In [ ]:
# 特徴量選択（f_regressionを使って上位の特徴を選択）
def select_features(X, y, k=30):
  # 数値型以外のカラムを削除
  X_numeric = X.select_dtypes(include=np.number)
  # もし数値型カラムがkより少ない場合はkを調整
  k = min(k, X_numeric.shape[1])

  selector = SelectKBest(score_func=f_regression, k=k)
  X_selected = selector.fit_transform(X_numeric, y)
  # 選択された特徴のインデックスを取得
  selected_indices = selector.get_support(indices=True)
  selected_features = X_numeric.columns[selected_indices] # X_numericのカラム名を使用
  print(f"選択された特徴量 ({k}個): {selected_features.tolist()}")
  # 選択された特徴をデータフレームとして返す
  return X_numeric.iloc[:, selected_indices] # X_numericから選択

# PCAで次元削減
def apply_pca(X, n_components=15):
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  pca = PCA(n_components=n_components)
  X_pca = pca.fit_transform(X_scaled)
  # 説明された分散の⽐率を確認
  explained_variance = pca.explained_variance_ratio_.sum()
  print(f"PCAにより{n_components}成分で{explained_variance:.2%}の分散を説明")
  return X_pca, pca, scaler

# 特徴量選択と次元削減を適⽤
X_selected = select_features(X, y, k=30)
X_pca, pca, pca_scaler = apply_pca(X_selected, n_components=15)
# 相関ヒートマップ（選択された特徴量）
plt.figure(figsize=(15, 12))
# X_selected が DataFrame であることを確認
if isinstance(X_selected, pd.DataFrame):
    correlation = X_selected.corr()
    mask = np.triu(np.ones_like(correlation, dtype=bool))
    sns.heatmap(correlation, mask=mask, annot=False, cmap='coolwarm', linewidths=0.5)
    plt.title('選択された特徴量の相関ヒートマップ')
    plt.tight_layout()
    plt.savefig('feature_correlation.png')
    plt.close()
else:
    print("X_selected is not a DataFrame, cannot generate correlation heatmap.")

選択された特徴量 (30個): ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_nuclear', 'generation_other', 'generation_other_renewable', 'generation_solar', 'generation_waste', 'generation_wind_onshore', 'total_load_actual', 'valencia_humidity', 'valencia_wind_speed', 'valencia_wind_deg', 'madrid_wind_speed', 'madrid_clouds_all', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_wind_deg', 'bilbao_clouds_all', 'barcelona_wind_speed', 'barcelona_wind_deg', 'seville_pressure', 'seville_humidity', 'seville_wind_speed', 'seville_wind_deg', 'seville_clouds_all']
PCAにより15成分で80.65%の分散を説明


## 時系列データの準備

In [ ]:
def create_sequences(X, y, seq_length=24):
    """
    時系列データのシーケンスを作成
    seq_length: 何時間前までのデータを使うか（例：24時間）
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# 時系列分割（単純に過去と未来で分けるのではなく、時系列分割を使用）
# 全体の80%をトレーニング、10%を検証、10%をテストに使用
train_size = int(len(X_pca) * 0.8)
val_size = int(len(X_pca) * 0.1)

# スケーリング
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))

# トレーニング、検証、テストデータの分割
X_train = X_pca[:train_size]
X_val = X_pca[train_size:train_size+val_size]
X_test = X_pca[train_size+val_size:]

y_train = y_scaled[:train_size]
y_val = y_scaled[train_size:train_size+val_size]
y_test = y_scaled[train_size+val_size:]

# シーケンスデータの作成
seq_length = 24  # 24時間（1日）のデータを使って予測
X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

print(f"トレーニングデータ形状: {X_train_seq.shape}, {y_train_seq.shape}")
print(f"検証データ形状: {X_val_seq.shape}, {y_val_seq.shape}")
print(f"テストデータ形状: {X_test_seq.shape}, {y_test_seq.shape}")

トレーニングデータ形状: (21000, 24, 15), (21000, 1)
検証データ形状: (2604, 24, 15), (2604, 1)
テストデータ形状: (2604, 24, 15), (2604, 1)


## モデル構築と学習

### LSTMモデル

In [ ]:
def build_lstm_model(seq_length, n_features):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        LSTM(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# LSTMモデルの構築
lstm_model = build_lstm_model(seq_length, X_train_seq.shape[2])
print(lstm_model.summary())

# コールバックの設定
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    verbose=1,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'lstm_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

# モデルの学習
lstm_history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping, checkpoint, reduce_lr],
    verbose=1
)

# 学習履歴のプロット
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(lstm_history.history['loss'], label='Training Loss')
plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
plt.title('LSTM - Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(lstm_history.history['mae'], label='Training MAE')
plt.plot(lstm_history.history['val_mae'], label='Validation MAE')
plt.title('LSTM - Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.savefig('lstm_training_history.png')
plt.close()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 24, 64)         │        20,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,225 (485.25 KB)

 Trainable params: 123,841 (483.75 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.2284 - mae: 0.3458 - val_loss: 0.0092 - val_mae: 0.0789 - learning_rate: 0.0010
Epoch 2/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0248 - mae: 0.1243
Epoch 2: val_loss improved from 0.00921 to 0.00605, saving model to lstm_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.0248 - mae: 0.1242 - val_loss: 0.0061 - val_mae: 0.0632 - learning_rate: 0.0010
Epoch 3/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0161 - mae: 0.0995
Epoch 3: val_loss did not improve from 0.00605
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0161 - mae: 0.0995 - val_loss: 0.0068 - val_mae: 0.0680 - learning_rate: 0.0010
Epoch 4/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0136 - mae: 0.0919
Epoch 4: val_loss did not improve from 0.00605
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0136 - mae: 0.0919 - val_loss: 0.0082 - val_mae: 0.0766 - learning_rate: 0.0010
Epoch 5/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0125 - mae: 0.0876
Epoch 5: val_loss improved from 0.00605 to 0.00529, saving model to lstm_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0125 - mae: 0.0876 - val_loss: 0.0053 - val_mae: 0.0591 - learning_rate: 0.0010
Epoch 6/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0109 - mae: 0.0820
Epoch 6: val_loss did not improve from 0.00529
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0109 - mae: 0.0820 - val_loss: 0.0062 - val_mae: 0.0646 - learning_rate: 0.0010
Epoch 7/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0103 - mae: 0.0794
Epoch 7: val_loss did not improve from 0.00529
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0103 - mae: 0.0793 - val_loss: 0.0056 - val_mae: 0.0616 - learning_rate: 0.0010
Epoch 8/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0097 - mae: 0.0775
Epoch 8: val_loss did not improve from 0.00529
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0097 - mae: 0.0775 - val_loss: 0.0065 - val_mae: 0.0666 - learning_rate: 0.0010
Epoch 9/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0742
Epoch

329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0089 - mae: 0.0742 - val_loss: 0.0050 - val_mae: 0.0583 - learning_rate: 0.0010
Epoch 10/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0713
Epoch 10: val_loss did not improve from 0.00500
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0082 - mae: 0.0713 - val_loss: 0.0070 - val_mae: 0.0703 - learning_rate: 0.0010
Epoch 11/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0083 - mae: 0.0717
Epoch 11: val_loss improved from 0.00500 to 0.00447, saving model to lstm_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0083 - mae: 0.0717 - val_loss: 0.0045 - val_mae: 0.0564 - learning_rate: 0.0010
Epoch 12/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0084 - mae: 0.0719
Epoch 12: val_loss did not improve from 0.00447
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0084 - mae: 0.0719 - val_loss: 0.0070 - val_mae: 0.0723 - learning_rate: 0.0010
Epoch 13/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0073 - mae: 0.0675
Epoch 13: val_loss did not improve from 0.00447
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0073 - mae: 0.0675 - val_loss: 0.0073 - val_mae: 0.0742 - learning_rate: 0.0010
Epoch 14/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0069 - mae: 0.0652
Epoch 14: val_loss did not improve from 0.00447
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0069 - mae: 0.0652 - val_loss: 0.0070 - val_mae: 0.0718 - learning_rate: 0.0010
Epoch 15/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0062 - mae: 0.06

### GRU モデル（比較用）

In [ ]:
def build_gru_model(seq_length, n_features):
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        GRU(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# GRUモデルの構築と学習
gru_model = build_gru_model(seq_length, X_train_seq.shape[2])
print(gru_model.summary())

gru_history = gru_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
        ModelCheckpoint('gru_model.h5', monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
    ],
    verbose=1
)

# 学習履歴のプロット
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(gru_history.history['loss'], label='Training Loss')
plt.plot(gru_history.history['val_loss'], label='Validation Loss')
plt.title('GRU - Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(gru_history.history['mae'], label='Training MAE')
plt.plot(gru_history.history['val_mae'], label='Validation MAE')
plt.title('GRU - Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.savefig('gru_training_history.png')
plt.close()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 24, 64)         │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,977 (371.00 KB)

 Trainable params: 94,593 (369.50 KB)

 Non-trainable params: 384 (1.50 KB)

None
Epoch 1/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3987 - mae: 0.4635
Epoch 1: val_loss improved from inf to 0.01095, saving model to gru_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.3957 - mae: 0.4613 - val_loss: 0.0110 - val_mae: 0.0831 - learning_rate: 0.0010
Epoch 2/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0370 - mae: 0.1509
Epoch 2: val_loss improved from 0.01095 to 0.00555, saving model to gru_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0370 - mae: 0.1508 - val_loss: 0.0055 - val_mae: 0.0574 - learning_rate: 0.0010
Epoch 3/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0207 - mae: 0.1123
Epoch 3: val_loss did not improve from 0.00555
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0207 - mae: 0.1122 - val_loss: 0.0081 - val_mae: 0.0758 - learning_rate: 0.0010
Epoch 4/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0179 - mae: 0.1050
Epoch 4: val_loss did not improve from 0.00555
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0179 - mae: 0.1050 - val_loss: 0.0065 - val_mae: 0.0658 - learning_rate: 0.0010
Epoch 5/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0163 - mae: 0.1002
Epoch 5: val_loss improved from 0.00555 to 0.00476, saving model to gru_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0163 - mae: 0.1002 - val_loss: 0.0048 - val_mae: 0.0549 - learning_rate: 0.0010
Epoch 6/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0153 - mae: 0.0976 
Epoch 6: val_loss did not improve from 0.00476
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0153 - mae: 0.0976 - val_loss: 0.0069 - val_mae: 0.0684 - learning_rate: 0.0010
Epoch 7/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0143 - mae: 0.0939
Epoch 7: val_loss did not improve from 0.00476
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0143 - mae: 0.0939 - val_loss: 0.0048 - val_mae: 0.0544 - learning_rate: 0.0010
Epoch 8/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0135 - mae: 0.0908
Epoch 8: val_loss did not improve from 0.00476
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0135 - mae: 0.0908 - val_loss: 0.0057 - val_mae: 0.0614 - learning_rate: 0.0010
Epoch 9/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0126 - mae: 0.0884
Epoch

329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0091 - mae: 0.0740 - val_loss: 0.0045 - val_mae: 0.0551 - learning_rate: 1.0000e-04
Epoch 17/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0735
Epoch 17: val_loss did not improve from 0.00446
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0089 - mae: 0.0735 - val_loss: 0.0048 - val_mae: 0.0575 - learning_rate: 1.0000e-04
Epoch 18/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0731
Epoch 18: val_loss did not improve from 0.00446
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0089 - mae: 0.0731 - val_loss: 0.0053 - val_mae: 0.0605 - learning_rate: 1.0000e-04
Epoch 19/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0088 - mae: 0.0727
Epoch 19: val_loss did not improve from 0.00446
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0088 - mae: 0.0727 - val_loss: 0.0047 - val_mae: 0.0569 - learning_rate: 1.0000e-04
Epoch 20/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.

## モデル評価と予測

In [ ]:
def evaluate_model(model, X_test, y_test, scaler, model_name):
    # 予測
    y_pred = model.predict(X_test)

    # スケーリングを元に戻す
    y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_orig = scaler.inverse_transform(y_pred).flatten()

    # 評価指標の計算
    mae = mean_absolute_error(y_test_orig, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
    r2 = r2_score(y_test_orig, y_pred_orig)

    print(f"\n{model_name} モデルの評価:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")

    # 予測と実際の値のプロット
    plt.figure(figsize=(15, 6))
    plt.plot(y_test_orig, label='実際の価格', color='blue')
    plt.plot(y_pred_orig, label='予測価格', color='red', alpha=0.7)
    plt.title(f'{model_name} - 予測 vs 実際の価格')
    plt.xlabel('時間')
    plt.ylabel('価格')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{model_name.lower()}_predictions.png')
    plt.close()

    return y_test_orig, y_pred_orig

# モデルの評価
lstm_y_test, lstm_y_pred = evaluate_model(lstm_model, X_test_seq, y_test_seq, y_scaler, 'LSTM')
gru_y_test, gru_y_pred = evaluate_model(gru_model, X_test_seq, y_test_seq, y_scaler, 'GRU')

# モデル比較
plt.figure(figsize=(15, 6))
plt.plot(lstm_y_test[:100], label='実際の価格', color='blue')
plt.plot(lstm_y_pred[:100], label='LSTM 予測', color='red', alpha=0.7)
plt.plot(gru_y_pred[:100], label='GRU 予測', color='green', alpha=0.7)
plt.title('モデル比較 (最初の100時間)')
plt.xlabel('時間')
plt.ylabel('価格')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('model_comparison.png')
plt.close()

82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

LSTM モデルの評価:
MAE: 10.63
RMSE: 13.43
R²: -0.3365
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

GRU モデルの評価:
MAE: 8.54
RMSE: 11.48
R²: 0.0240


## 特徴量重要度の分析

In [ ]:
# 特徴量重要度の可視化（SelectKBestの結果）
def plot_feature_importance(X, y, top_n=20):
    # 数値型以外のカラムを削除
    X_numeric = X.select_dtypes(include=np.number)

    selector = SelectKBest(score_func=f_regression, k='all')
    # 数値型カラムのみを使ってfitを行う
    selector.fit(X_numeric, y)

    # 特徴量のスコアを取得
    feature_scores = pd.DataFrame({
        'Feature': X_numeric.columns, # X_numericのカラム名を使用
        'Score': selector.scores_
    })

    # スコア順にソート
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    # 上位n個の特徴量をプロット
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Score', y='Feature', data=feature_scores.head(top_n))
    plt.title(f'上位{top_n}個の重要な特徴量')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()

    return feature_scores

# 特徴量重要度のプロット
# XがDataFrameであり、数値型以外のカラムを含んでいる可能性があるため、関数内で処理します
importance = plot_feature_importance(X, y, top_n=20)
print("\n上位20の重要な特徴量:")
print(importance.head(20))


上位20の重要な特徴量:
                                        Feature        Score
3                   generation_fossil_hard_coal  7410.283193
1          generation_fossil_brown_coal/lignite  4315.279936
14                            total_load_actual  4191.590208
2                         generation_fossil_gas  4151.326152
5   generation_hydro_pumped_storage_consumption  2642.416186
6    generation_hydro_run_of_river_and_poundage  1905.413869
4                         generation_fossil_oil  1735.021199
32                            madrid_wind_speed  1515.924859
0                            generation_biomass  1159.447579
42                              bilbao_pressure  1094.408617
66                             seville_pressure   799.616985
69                             seville_wind_deg   731.785895
44                            bilbao_wind_speed   685.479126
49                            bilbao_clouds_all   569.759390
56                         barcelona_wind_speed   551.164363
20        

## 時系列交差検証

In [ ]:
# 時系列交差検証
def time_series_cv_evaluation(X, y, seq_length=24, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    mae_scores = []
    rmse_scores = []
    fold = 1

    for train_index, test_index in tscv.split(X):
        print(f"\n評価 fold {fold}/{n_splits}")

        # データの分割
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # スケーリング
        y_scaler_fold = MinMaxScaler()
        y_train_scaled = y_scaler_fold.fit_transform(y_train_fold.reshape(-1, 1))
        y_test_scaled = y_scaler_fold.transform(y_test_fold.reshape(-1, 1))

        # シーケンスデータの作成
        X_train_seq_fold, y_train_seq_fold = create_sequences(X_train_fold, y_train_scaled, seq_length)
        X_test_seq_fold, y_test_seq_fold = create_sequences(X_test_fold, y_test_scaled, seq_length)

        if len(X_train_seq_fold) == 0 or len(X_test_seq_fold) == 0:
            print("シーケンスデータが空です")
            continue

        # モデル構築と学習
        model = build_lstm_model(seq_length, X_train_seq_fold.shape[2])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

        model.fit(
            X_train_seq_fold, y_train_seq_fold,
            epochs=50,
            batch_size=64,
            validation_split=0.2,
            callbacks=[early_stopping],
            verbose=0
        )

        # 評価
        y_pred_fold = model.predict(X_test_seq_fold)

        # スケーリングを元に戻す
        y_test_orig_fold = y_scaler_fold.inverse_transform(y_test_seq_fold.reshape(-1, 1)).flatten()
        y_pred_orig_fold = y_scaler_fold.inverse_transform(y_pred_fold).flatten()

        # 評価指標の計算
        mae = mean_absolute_error(y_test_orig_fold, y_pred_orig_fold)
        rmse = np.sqrt(mean_squared_error(y_test_orig_fold, y_pred_orig_fold))

        mae_scores.append(mae)
        rmse_scores.append(rmse)

        print(f"Fold {fold} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
        fold += 1

    # 結果をまとめる
    print("\n時系列交差検証の結果:")
    print(f"平均 MAE: {np.mean(mae_scores):.2f} (±{np.std(mae_scores):.2f})")
    print(f"平均 RMSE: {np.mean(rmse_scores):.2f} (±{np.std(rmse_scores):.2f})")

    return mae_scores, rmse_scores

# 時系列交差検証の実行
mae_cv, rmse_cv = time_series_cv_evaluation(X_pca, y_scaled, seq_length=24, n_splits=3)


評価 fold 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Fold 1 - MAE: 0.20, RMSE: 0.25

評価 fold 2/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Fold 2 - MAE: 0.14, RMSE: 0.18

評価 fold 3/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Fold 3 - MAE: 0.10, RMSE: 0.13

時系列交差検証の結果:
平均 MAE: 0.15 (±0.04)
平均 RMSE: 0.19 (±0.05)


## モデルとパイプラインの保存

In [ ]:
import pickle

# モデル保存のためのパイプラインを作成
pipeline = {
    'pca': pca,
    'pca_scaler': pca_scaler,
    'y_scaler': y_scaler,
    'selected_features': X_selected.columns.tolist(),
    'seq_length': seq_length
}

# パイプラインを保存
pipeline_save_path = data_path + 'price_prediction_pipeline.pkl'
with open(pipeline_save_path, 'wb') as f:
    pickle.dump(pipeline, f)

# 予測パイプラインの使用例
def predict_price(model, new_data, pipeline):
    """
    新しいデータで価格を予測する

    Parameters:
    - model: 訓練済みモデル
    - new_data: 予測したいデータ（必要な特徴量を含むDataFrame）
    - pipeline: 前処理パイプライン

    Returns:
    - predicted_price: 予測された価格
    """
    # パイプラインから必要な要素を取得
    pca = pipeline['pca']
    pca_scaler = pipeline['pca_scaler']
    y_scaler = pipeline['y_scaler']
    selected_features = pipeline['selected_features']
    seq_length = pipeline['seq_length']

    # 必要な特徴量のみを選択
    selected_data = new_data[selected_features]

    # スケーリングとPCA変換
    scaled_data = pca_scaler.transform(selected_data)
    pca_data = pca.transform(scaled_data)

    # シーケンスデータの作成（最新のseq_length分のデータを使用）
    if len(pca_data) >= seq_length:
        sequence = pca_data[-seq_length:].reshape(1, seq_length, -1)

        # 予測
        scaled_prediction = model.predict(sequence)

        # スケーリングを元に戻す
        prediction = y_scaler.inverse_transform(scaled_prediction)[0, 0]

        return prediction
    else:
        raise ValueError(f"入力データが短すぎます。少なくとも{seq_length}点必要です。")

print("\nモデルと前処理パイプラインを保存しました。")
print("新しいデータに対する予測は 'predict_price' 関数を使用して行えます。")


モデルと前処理パイプラインを保存しました。
新しいデータに対する予測は 'predict_price' 関数を使用して行えます。


In [ ]:
import pandas as pd
import numpy as np
import os # osモジュールをインポート

# test_dfに対する推論
# test_IDが'spain_total'のデータのみを選択
test_df_spain = test_df[test_df['item_ID'] == 'spain_total'].copy()

# 欠損値処理 (trainと同じ処理を適用)
test_df_spain = test_df_spain.ffill()
test_df_spain = test_df_spain.bfill()

# 'item_ID' 列を削除
test_df_spain = test_df_spain.drop('item_ID', axis=1)

# 保存したパイプラインを読み込み
pipeline_load_path = data_path + 'price_prediction_pipeline.pkl'
if os.path.exists(pipeline_load_path): # ファイルが存在するか確認
    with open(pipeline_load_path, 'rb') as f:
        loaded_pipeline = pickle.load(f)

    # 必要な要素をパイプラインから取得
    loaded_pca = loaded_pipeline['pca']
    loaded_pca_scaler = loaded_pipeline['pca_scaler']
    loaded_y_scaler = loaded_pipeline['y_scaler']
    loaded_selected_features = loaded_pipeline['selected_features']
    loaded_seq_length = loaded_pipeline['seq_length']

else:
    print(f"エラー: パイプラインファイル '{pipeline_load_path}' が見つかりません。")
    # パイプラインがロードできない場合は処理を中断
    loaded_pipeline = None


if loaded_pipeline is not None:
    # モデルをロード (LSTM モデルを使用)
    # 保存されたモデルファイル名が 'lstm_model.h5' であることを確認
    lstm_model_path = data_path + 'lstm_model.h5' # h5形式の場合
    # lstm_model_path = data_path + 'lstm_savedmodel' # SavedModel形式の場合
    print(lstm_model_path)
    try:
        # h5形式でmseエラーが出る場合のみ custom_objects を試す
        # ModelCheckpointでSavedModel形式で保存している場合は、パスを修正し custom_objects は不要
        loaded_model = tf.keras.models.load_model(lstm_model_path, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
        # モデルロード時に発生する警告に対応するコードは削除またはコメントアウト
        # if loaded_model.compile_metrics is None:
        #     print("警告: モデルのコンパイルメトリクスはロードされましたが、ビルドされていません。推論には影響しない可能性があります。")

    except Exception as e:
        print(f"モデルのロードに失敗しました: {e}")
        print("モデルファイルが存在し、正しく保存されているか、またはロードパスが正しいか確認してください。")
        loaded_model = None # エラーが発生した場合はNoneを設定

    if loaded_model is not None:
        # テストデータに対して前処理を適用
        # 必要な特徴量のみを選択
        test_data_selected = test_df_spain[loaded_selected_features]

        # スケーリングとPCA変換
        test_data_scaled = loaded_pca_scaler.transform(test_data_selected)
        test_data_pca = loaded_pca.transform(test_data_scaled)

        # 推論用データセットの作成
        # 学習データの最後の部分とテストデータを結合
        # Note: X は学習データ全体の選択済み特徴量（PCA適用前）です。
        # ここで結合するのはPCA適用後のデータにする必要があります。
        # したがって、学習データ全体のPCA適用後のデータが必要です。
        # 元のコードで X_pca が学習データ全体のPCA適用後のデータとして生成されています。
        # X_pca がまだ利用可能な状態でこのブロックが実行されている必要があります。
        if 'X_pca' not in locals() and 'X_pca' not in globals():
            print("エラー: 学習データに対するPCA適用結果 (X_pca) が見つかりません。前のセルが実行されているか確認してください。")
            predictions = np.array([]) # エラーを回避
        else:
            combined_data_pca = np.concatenate([X_pca, test_data_pca])

            # テストデータの予測に必要な入力シーケンスを作成
            X_inference = []
            # combined_data_pca のインデックスは 0 から len(combined_data_pca) - 1
            # テストデータに対応する部分は len(X_pca) から len(combined_data_pca) - 1

            start_index_test_in_combined = len(X_pca)

            for i in range(len(test_data_pca)):
                # 現在予測したいテストデータの時刻に対応する combined_data_pca におけるインデックス
                current_index_in_combined = start_index_test_in_combined + i

                # 予測に使用する過去のシーケンスの開始インデックス
                seq_start_index = current_index_in_combined - loaded_seq_length
                # 予測に使用する過去のシーケンスの終了インデックス (この時刻を含まない)
                seq_end_index = current_index_in_combined

                # seq_start_index が0未満になる場合、これは学習データの最初よりも過去を参照しようとしている場合です。
                # 結合データを使っているため、start_index_test_in_combined >= loaded_seq_length であれば問題ありません。
                # データセットのサイズを確認し、必要であればエラーハンドリングを追加できますが、
                # 一般的な時系列予測ではテストデータは学習データの直後に続くため、通常は問題ありません。
                if seq_start_index < 0:
                    # データセットのサイズが小さすぎる場合に発生する可能性
                    print(f"警告: テストデータの最初の予測シーケンスが学習データの最初よりも過去を参照しようとしています (開始インデックス {seq_start_index})")
                    # 実際には seq_start_index は常に >= 0 となるはずですが、念のため警告を出力
                    # より堅牢にするなら、ここでエラーにするか、特別な初期シーケンス生成ロジックが必要
                    pass # エラーにせず警告に留める

                # 有効なシーケンスが取得できるか確認
                if seq_start_index < 0 or seq_end_index - seq_start_index != loaded_seq_length:
                     print(f"エラー: 無効なシーケンスインデックスが生成されました (start: {seq_start_index}, end: {seq_end_index})。データ長またはseq_lengthを確認してください。")
                     X_inference = [] # エラーが発生したら空にしてループを抜ける
                     break # ループを中断

                input_sequence = combined_data_pca[seq_start_index : seq_end_index]
                X_inference.append(input_sequence)


            # X_inference が空になる可能性がある場合、処理をスキップ
            if not X_inference:
                print("エラー: 予測のための入力シーケンスが作成できませんでした。データ長またはseq_lengthを確認してください。")
                predictions = np.array([]) # 空の配列を設定してエラーを回避
            else:
                X_inference = np.array(X_inference)
                print(f"推論データ形状: {X_inference.shape}")

                # 予測を実行
                scaled_predictions = loaded_model.predict(X_inference)

                # スケーリングを元に戻す
                predictions = loaded_y_scaler.inverse_transform(scaled_predictions).flatten()

            # 提出用ファイルの作成
            # sample_submission_dfを参考にする
            # 提出形式: id, price_actual
            # test_dfの'time'と'item_ID'からidを作成し、予測結果と結合する

            if len(predictions) > 0: # 予測結果がある場合のみ提出ファイルを作成
                # 'id' 列を作成 (test_df_spainのインデックスを使用)
                # ここで AttributeError を修正するために、strftime を各要素に適用します
                submission_df = pd.DataFrame({
                    'id': [ts.strftime('%Y-%m-%d %H:%M:%S') + '_spain_total' for ts in test_df_spain.index],
                    'price_actual': predictions
                })

                # sample_submit.csv の形式 (header=None) に合わせる
                # headerは出力しない
                # sample_submit.csv は 'YYYY-MM-DD HH:MM:SS_spain_total' のID形式なので、それに合わせる
                # test_dfのindexはdatetime型なので、strftimeで指定の形式に変換

                # 提出ファイルの保存パス
                submission_path = data_path + 'submission.csv'

                # ヘッダーなし、インデックスなしでCSVとして保存
                submission_df.to_csv(submission_path, header=False, index=False)

                print(f"\n推論結果を '{submission_path}' に保存しました。")
                print("提出ファイルの先頭:")
                # ファイルを読み込んで表示する前に、ファイルが存在するか確認
                if os.path.exists(submission_path):
                    display(pd.read_csv(submission_path, header=None).head())
                else:
                    print(f"エラー: '{submission_path}' ファイルが生成されませんでした。")
            else:
                print("\n予測結果が生成されなかったため、提出ファイルは作成されませんでした。")

    else:
        print("\nモデルがロードされなかったため、推論を実行できません。")
else:
    print("\nパイプラインがロードされなかったため、推論を実行できません。")

/content/drive/MyDrive/ML/Signate_1634/lstm_model.h5
推論データ形状: (8760, 24, 15)
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

推論結果を '/content/drive/MyDrive/ML/Signate_1634/submission.csv' に保存しました。
提出ファイルの先頭:


,0,1
0,2018-01-01 00:00:00_spain_total,49.507435
1,2018-01-01 01:00:00_spain_total,46.879658
2,2018-01-01 02:00:00_spain_total,44.471542
3,2018-01-01 03:00:00_spain_total,43.089245
4,2018-01-01 04:00:00_spain_total,41.699215
